In [67]:
!pip install medmnist
import tensorflow as tf
import tensorflow_probability as tfp
import medmnist as mn
import numpy as np
import time
import sys

from google.colab import drive
drive.mount('/content/drive')

from matplotlib import pyplot as plt
from pprint import pprint

BATCH_SIZE = 32
N_LATENT = 128
EPOCHS = 100
SIZE = 100

PROJECT_DIR = "/content/drive/My Drive/ADL_Miniproject/"
GEOMETRIC_MODEL = "Geometric"
CVAE_MODEL = "CVAE"
DCGAN_MODEL = "DCGAN"
CVAEDCGAN_MODEL = "CVAEDCGAN"
CLASSIFIER_MODEL = "Classifier"

RESULTS = {
    "Classifier": {
        "testing_acc": {
            "Geometric": {},
            "CVAE": {},
            "DCGAN": {},
            "CVAEDCGAN": {}
        }
    },
    "Predictions": {
        "generated_labels": {
            "Geometric": {},
            "CVAE": {},
            "DCGAN": {},
            "CVAEDCGAN": {}
        },
        "generated_confidance": {
            "Geometric": {},
            "CVAE": {},
            "DCGAN": {},
            "CVAEDCGAN": {}
        }
    }
}

def get_model_dir(model: str):
    return f"{PROJECT_DIR}{model}/Models/"

def get_generated_dir(model: str):
    return f"{PROJECT_DIR}{model}/GeneratedImages/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
DATASETS = {}
for label, (dataset, n_labels) in {
		"PathMNIST": 		(mn.PathMNIST, 9), # 9 mulit-class
		"DermaMNIST": 		(mn.DermaMNIST, 7), # 7 multi-calss
		"OCTMNIST": 		(mn.OCTMNIST, 4),  # 4 multi-class
		"PneumoniaMNIST": 	(mn.PneumoniaMNIST, 2), # 2 binary-calss
		"BreastMNIST": 		(mn.BreastMNIST, 2), # 2 binary-class
		"OrganAMNIST": 		(mn.OrganAMNIST, 11), # 11 multi-class
		"OrganCMNIST": 		(mn.OrganCMNIST, 11), # 11 multi-class
		"OrganSMNIST": 		(mn.OrganSMNIST, 11), # 11 multi-class
	}.items():
	train = dataset(split = "train", download = True)
	test = dataset(split = "test", download = False)	
	DATASETS[label] = (
		(train.imgs, train.labels), 
		(test.imgs, test.labels),
		n_labels
	)

Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/dermamnist.npz
Using downloaded and verified file: /root/.medmnist/octmnist.npz
Using downloaded and verified file: /root/.medmnist/pneumoniamnist.npz
Using downloaded and verified file: /root/.medmnist/breastmnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organcmnist.npz
Using downloaded and verified file: /root/.medmnist/organsmnist.npz


In [69]:
def shuffle_images(images, batch_size):
    return (
        tf.data.Dataset
            .from_tensor_slices(images)
            .shuffle(len(images))
            .batch(batch_size)
    )

def preprocess_geometric(data, labels, size):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range = 90, 
        width_shift_range = 0.2, 
        height_shift_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True
    )
    datagen.fit(data)
    return next(datagen.flow(data, labels, batch_size = size, shuffle = False))

def preprocess_normalize(data):
    if len(data.shape) < 4:
        data = data.reshape(*data.shape, 1)
    return data.astype('float32') / 255.

def preprocess_datasets_generator(mnist = False):
    if mnist == True:
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
        x_train = preprocess_normalize(x_train)
        x_test = preprocess_normalize(x_test)

        # train_dataset = shuffle_images(x_train, BATCH_SIZE)
        # test_dataset = shuffle_images(x_test, BATCH_SIZE)

        yield (x_train, y_train), (x_test, y_test)
    else:
        for label, ((x_train, y_train), (x_test, y_test), n_labels) in DATASETS.items():
            x_train = preprocess_normalize(x_train)
            x_test = preprocess_normalize(x_test)

            # train_dataset = shuffle_images(x_train, BATCH_SIZE)
            # test_dataset = shuffle_images(x_test, BATCH_SIZE)
            yield label, (x_train, y_train), (x_test, y_test), n_labels

In [70]:
class Sampling(tf.keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [71]:
def get_encoder(input_shape: tuple, output_shape: int):
    model = tf.keras.Sequential(
        [
         tf.keras.Input(shape = input_shape),
         tf.keras.layers.Conv2D(32, kernel_size = 5, strides = 2, padding = "same"),
         tf.keras.layers.LeakyReLU(alpha = 0.2),
         tf.keras.layers.Conv2D(64, kernel_size = 5, strides = 2, padding = "same"),
         tf.keras.layers.LeakyReLU(alpha = 0.2),
         tf.keras.layers.Conv2D(128, kernel_size = 5, strides = 2, padding = "same"),
         tf.keras.layers.LeakyReLU(alpha = 0.2),
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dense(output_shape)
        ],
        name = "encoder"
    )
    z_mean = tf.keras.layers.Dense(output_shape, name = "z_mean")(model.output)
    z_log_var = tf.keras.layers.Dense(output_shape, name = "z_log_var")(model.output)
    z = Sampling()([z_mean, z_log_var])
    model = tf.keras.Model(model.input, [z_mean, z_log_var, z], name = "encoder")
    model.summary()
    tf.keras.utils.plot_model(model, to_file = f"{PROJECT_DIR}encoder_model_plot.png", show_shapes=True)
    return model

def get_decoder(input_shape: tuple, output_shape: int):
    model = tf.keras.Sequential(
      [
        tf.keras.Input(shape = input_shape),
        tf.keras.layers.Dense(7 * 7 * 256),
        tf.keras.layers.Reshape((7, 7, 256)),
        tf.keras.layers.BatchNormalization(epsilon = 1e-5),
        tf.keras.layers.LeakyReLU(alpha = 0.2),
        tf.keras.layers.Conv2DTranspose(128, kernel_size = 5, strides = 2, padding = "same"),
        tf.keras.layers.BatchNormalization(epsilon = 1e-5),
        tf.keras.layers.LeakyReLU(alpha = 0.2),
        tf.keras.layers.Conv2DTranspose(64, kernel_size = 5, strides = 2, padding = "same"),
        tf.keras.layers.BatchNormalization(epsilon = 1e-5),
        tf.keras.layers.LeakyReLU(alpha = 0.2),
        tf.keras.layers.Conv2DTranspose(32, kernel_size = 5, strides = 1, padding = "same"),
        tf.keras.layers.BatchNormalization(epsilon = 1e-5),
        tf.keras.layers.LeakyReLU(alpha = 0.2),
        tf.keras.layers.Conv2DTranspose(output_shape, kernel_size = 2, padding = "same", activation = "sigmoid")
      ],
      name = "decoder"
    )
    model.summary()
    tf.keras.utils.plot_model(model, to_file = f"{PROJECT_DIR}decoder_model_plot.png", show_shapes=True)
    return model

def get_generator(input_shape: tuple, output_shape: int):
    model = get_decoder(input_shape, output_shape)
    model.name = "generator"
    return model

def get_discriminator(input_shape: tuple, output_shape: int):
    model = tf.keras.Sequential(
      [
       tf.keras.Input(shape = input_shape),
       tf.keras.layers.Conv2D(64, kernel_size = 5, strides = 2, padding = "same"),
       tf.keras.layers.LeakyReLU(alpha = 0.2),
       tf.keras.layers.Conv2D(128, kernel_size = 5, strides = 2, padding = "same"),
       tf.keras.layers.BatchNormalization(epsilon = 1e-5),
       tf.keras.layers.LeakyReLU(alpha = 0.2),
       tf.keras.layers.Conv2D(256, kernel_size = 5, strides = 2, padding = "same"),
       tf.keras.layers.BatchNormalization(epsilon = 1e-5),
       tf.keras.layers.LeakyReLU(alpha = 0.2),
       tf.keras.layers.Conv2D(256, kernel_size = 5, strides = 2, padding = "same"),
      ],
      name = "discriminator"
    )
    x = tf.keras.layers.BatchNormalization(epsilon = 1e-5)(model.output)
    x = tf.keras.layers.LeakyReLU(alpha = 0.2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(output_shape, activation = "sigmoid")(x)
    
    model = tf.keras.Model(model.input, [x, model.output])
    model.summary()
    tf.keras.utils.plot_model(model, to_file = f"{PROJECT_DIR}discriminator_model_plot.png", show_shapes=True)
    return model

def get_classifier(input_shape: tuple, output_shape: int):
    model = tf.keras.Sequential(
        [
         tf.keras.Input(shape = input_shape),
         tf.keras.layers.Conv2D(32, kernel_size = 5, strides = 2, padding = "same", kernel_regularizer = tf.keras.regularizers.l2(1e-4)),
         tf.keras.layers.LeakyReLU(alpha = 0.2),
         tf.keras.layers.Conv2D(64, kernel_size = 5, strides = 2, padding = "same", kernel_regularizer = tf.keras.regularizers.l2(1e-4)),
         tf.keras.layers.LeakyReLU(alpha = 0.2),
         tf.keras.layers.Conv2D(128, kernel_size = 5, strides = 2, padding = "same", kernel_regularizer = tf.keras.regularizers.l2(1e-4)),
         tf.keras.layers.LeakyReLU(alpha = 0.2),
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.2),
         tf.keras.layers.Dense(256, kernel_regularizer = tf.keras.regularizers.l2(1e-4)),
         tf.keras.layers.LeakyReLU(alpha = 0.2),
         tf.keras.layers.Dense(output_shape),
         tf.keras.layers.Softmax()
        ],
        name = "classifier"
    )   
    model.summary()
    tf.keras.utils.plot_model(model, to_file = f"{PROJECT_DIR}classifier_model_plot.png", show_shapes=True)
    return model

In [7]:
class CVAE(tf.keras.Model):
    """docstring for CVAE"""
    def __init__(self, encoder, decoder, latent_dim, **kwargs):
        super(CVAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.latent_dim = latent_dim

    def compile(self, optimizer, loss_fn):
        super(CVAE, self).compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.loss_metric = tf.keras.metrics.Mean(name = "loss")

    def call(self, inputs):
        _, _, z = self.encoder(inputs)
        return self.decoder(z)

    def get_config(self):
        return {
            "encoder": self.encoder,
            "decoder": self.decoder,
            "latent_dim": self.latent_dim
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)

    @property
    def metrics(self):
        return [
            self.loss_metric,
        ]

    @tf.function
    def train_step(self, data):
        with tf.GradientTape() as tape:
            mean, logvar, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(self.loss_fn(data, reconstruction), axis = (1, 2)))
            kl_loss = tf.reduce_mean(tf.reduce_sum(-0.5 * (1 + logvar - tf.square(mean) - tf.exp(logvar)), axis = 1))    
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.loss_metric.update_state(total_loss)
        return {
            "loss": self.loss_metric.result()
        }


def plot_CVAE(model, data, latent_dim, label = "test", save = False):
    if (isinstance(data, np.ndarray)):
        data = tf.convert_to_tensor(data)
    num_images = data.shape[0]
    predictions = model(data, training = False)
    predictions_gen = model.decoder(tf.random.normal(shape = (num_images, latent_dim)), training = False)

    ncols = num_images
    nrows = 3

    fig, axs = plt.subplots(
        ncols = ncols, 
        nrows = nrows, 
        figsize = (2 * ncols, 2 * nrows)
    )
    for axi, (dat, lab) in enumerate(
        zip(
            [data, predictions, predictions_gen],
            ["data", "data recon", "gen"],
        )
    ):
        for ex in range(ncols):
            axs[axi, ex].matshow(
                dat.numpy()[ex].squeeze(), 
                cmap = plt.cm.gray, 
                vmin = 0, 
                vmax = 1
            )
            axs[axi, ex].axes.get_xaxis().set_ticks([])
            axs[axi, ex].axes.get_yaxis().set_ticks([])
        axs[axi, 0].set_ylabel(lab)
    if save:
        plt.savefig(f"{label}.png")
    else:
        plt.show()

class CVAEMonitor(tf.keras.callbacks.Callback):
    def __init__(self, images, num_img=3, latent_dim=128):
        self.images = images
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        plot_CVAE(self.model, self.images[:self.num_img], self.latent_dim)

In [19]:
class DCGAN(tf.keras.Model):
    """docstring for DCGAN"""
    def __init__(self, generator, discriminator, latent_dim, **kwargs):
        super(DCGAN, self).__init__(**kwargs)
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim

    def compile(self, g_optimizer, d_optimizer, loss_fn):
        super(DCGAN, self).compile()
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.loss_fn = loss_fn
        self.g_loss_metric = tf.keras.metrics.Mean(name = "g_loss")
        self.d_loss_metric = tf.keras.metrics.Mean(name = "d_loss")

    def call(self, inputs):
        return self.generator(inputs)

    def get_config(self):
        return {
            "generator": self.generator,
            "discriminator": self.discriminator,
            "latent_dim": self.latent_dim
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)

    @property
    def metrics(self):
        return [
            self.g_loss_metric,
            self.d_loss_metric
        ]

    @tf.function
    def train_step(self, data):
        batch_size = tf.shape(data)[0]
        random_latent_vectors = tf.random.normal(shape = (batch_size, self.latent_dim))

        generated_images = self.generator(random_latent_vectors)
        combined_images = tf.concat([generated_images, data], axis = 0)

        labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis = 0)
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        random_latent_vectors = tf.random.normal(shape = (batch_size, self.latent_dim))
        misleading_labels = tf.zeros((batch_size, 1))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss" : self.d_loss_metric.result(),
            "g_loss" : self.g_loss_metric.result()
        }

def plot_DCGAN(model, num_images, latent_dim, label = "test", save = False):
    predictions = model(tf.random.normal(shape = (num_images, latent_dim)), training = False)
    ncols = predictions.shape[0]
    nrows = 1

    fig, axs = plt.subplots(
        ncols = ncols, 
        nrows = nrows, 
        figsize = (2 * ncols, 2 * nrows)
    )
    for axi, (dat, lab) in enumerate(
        zip(
            [predictions],
            ["data recon"],
        )
    ):
        for ex in range(ncols):
            axs[ex].matshow(
                dat.numpy()[ex].squeeze(), 
                cmap = plt.cm.gray, 
                vmin = 0, 
                vmax = 1
            )
            axs[ex].axes.get_xaxis().set_ticks([])
            axs[ex].axes.get_yaxis().set_ticks([])
        axs[0].set_ylabel(lab)
    if save:
        plt.savefig(f"{label}.png")
    else:
        plt.show()

class DCGANMonitor(tf.keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        plot_DCGAN(self.model, self.num_img, self.latent_dim)

In [9]:
class CVAEDCGAN(tf.keras.Model):
    """docstring for CVAEDCGAN"""
    def __init__(self, encoder, decoder, discriminator, latent_dim, **kwargs):
        super(CVAEDCGAN, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.discriminator = discriminator
        self.latent_dim = latent_dim

    def compile(self, e_optimizer, g_optimizer, d_optimizer, loss_fn):
        super(CVAEDCGAN, self).compile()
        self.e_optimizer = e_optimizer
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.loss_fn = loss_fn
        self.e_loss_metric = tf.keras.metrics.Mean(name = "e_loss")
        self.g_loss_metric = tf.keras.metrics.Mean(name = "g_loss")
        self.d_loss_metric = tf.keras.metrics.Mean(name = "d_loss")

    def call(self, inputs):
        mean, logvar, z = self.encoder(inputs)
        return self.decoder(z)

    def get_config(self):
        return {
            "encoder": self.encoder,
            "decoder": self.decoder,
            "discriminator": self.discriminator,
            "latent_dim": self.latent_dim
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)

    @property
    def metrics(self):
        return [
            self.e_loss_metric,
            self.g_loss_metric,
            self.d_loss_metric
        ]

    @tf.function
    def train_step(self, data):
        with    tf.GradientTape() as e_tape, \
                tf.GradientTape() as g_tape, \
                tf.GradientTape() as d_tape:
            batch_size = tf.shape(data)[0]
            random_latent_vectors = tf.random.normal(shape = (batch_size, self.latent_dim))

            mean, logvar, z = self.encoder(data)
            q_z = tfp.distributions.MultivariateNormalDiag(
                loc = mean,
                scale_diag = logvar
            )
            p_z = tfp.distributions.MultivariateNormalDiag(
                loc = [0.] * self.latent_dim,
                scale_diag = [1.] * self.latent_dim
            )
            decoded_images = self.decoder(z)
            generated_images = self.decoder(random_latent_vectors)

            d_images, ld_images = self.discriminator(data)
            d_decoded_images, ld_decoded_images = self.discriminator(decoded_images)
            d_generated_images, ld_generated_images = self.discriminator(generated_images)

            disc_real_loss = self.loss_fn(tf.ones_like(d_images), d_images)
            disc_fake_loss = self.loss_fn(tf.zeros_like(d_generated_images), d_generated_images)
            gen_fake_loss = self.loss_fn(tf.ones_like(d_generated_images), d_generated_images)

            disc_layer_recon_loss = tf.reduce_mean(tf.reduce_mean(tf.math.square(ld_images - ld_decoded_images), axis = 0)) / 1e-3
            latent_loss = tf.reduce_mean(tf.maximum(tfp.distributions.kl_divergence(q_z, p_z), 0))

            e_loss = latent_loss + disc_layer_recon_loss
            g_loss = gen_fake_loss + disc_layer_recon_loss
            d_loss = disc_fake_loss + disc_real_loss

        e_grad = e_tape.gradient(e_loss, self.encoder.trainable_weights)
        g_grad = g_tape.gradient(g_loss, self.decoder.trainable_weights)
        d_grad = d_tape.gradient(d_loss, self.discriminator.trainable_weights)

        self.e_optimizer.apply_gradients(zip(e_grad, self.encoder.trainable_weights))
        self.g_optimizer.apply_gradients(zip(g_grad, self.decoder.trainable_weights))
        self.d_optimizer.apply_gradients(zip(d_grad, self.discriminator.trainable_weights))

        self.e_loss_metric.update_state(e_loss)
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "e_loss" : self.e_loss_metric.result(),
            "d_loss" : self.d_loss_metric.result(),
            "g_loss" : self.g_loss_metric.result()
        }

def plot_CVAEDCGAN(model, data, latent_dim, label = "test", save = False):
    if (isinstance(data, np.ndarray)):
        data = tf.convert_to_tensor(data)
    num_images = data.shape[0]
    predictions = model(data, training = False)
    predictions_gen = model.decoder(tf.random.normal(shape = (num_images, latent_dim)), training = False)

    ncols = num_images
    nrows = 3

    fig, axs = plt.subplots(
        ncols = ncols, 
        nrows = nrows, 
        figsize = (2 * ncols, 2 * nrows)
    )
    for axi, (dat, lab) in enumerate(
        zip(
            [data, predictions, predictions_gen],
            ["data", "data recon", "gen"],
        )
    ):
        for ex in range(ncols):
            axs[axi, ex].matshow(
                dat.numpy()[ex].squeeze(), 
                cmap = plt.cm.gray, 
                vmin = 0, 
                vmax = 1
            )
            axs[axi, ex].axes.get_xaxis().set_ticks([])
            axs[axi, ex].axes.get_yaxis().set_ticks([])
        axs[axi, 0].set_ylabel(lab)
    if save:
        plt.savefig(f"{label}.png")
    else:
        plt.show()

class CVAEDCGANMonitor(tf.keras.callbacks.Callback):
    def __init__(self, images, num_img=3, latent_dim=128):
        self.images = images
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        plot_CVAEDCGAN(self.model, self.images[:self.num_img], self.latent_dim)

In [74]:
for label, (train_dataset, train_labels), (test_dataset, test_labels), n_labels in preprocess_datasets_generator():
    # Geometric
    print(f"Geometric: {label}")
    generated_images_str = f"{get_generated_dir(GEOMETRIC_MODEL)}{label}_{SIZE}_n.npz"
    generated_images = preprocess_geometric(train_dataset, train_labels, SIZE)[0]
    generated_images = preprocess_normalize(generated_images)
    np.savez(generated_images_str, generated_images)
    
    # CVAE
    print(f"CVAE: {label}")
    model_str = f"{get_model_dir(CVAE_MODEL)}{label}_{EPOCHS}_epochs_{N_LATENT}_latent_dim_model.tf"
    generated_images_str = f"{get_generated_dir(CVAE_MODEL)}{label}_{EPOCHS}_epochs_{N_LATENT}_latent_dim_{SIZE}_n.npz"
    encoder = get_encoder(train_dataset.shape[1:], N_LATENT)
    decoder = get_decoder((N_LATENT,), train_dataset.shape[-1])
    cvae = CVAE(encoder, decoder, N_LATENT)
    cvae.compile(tf.keras.optimizers.Adam(1e-4), tf.keras.losses.binary_crossentropy)
    cvae.fit(train_dataset, epochs = EPOCHS, callbacks=[CVAEMonitor(test_dataset, num_img = 10, latent_dim = N_LATENT)], shuffle = True)
    cvae(tf.zeros((10, *train_dataset.shape[1:])))
    cvae.save(model_str, save_format="tf")
    generated_images = cvae.decoder(tf.random.normal(shape = (SIZE, N_LATENT)), training = False)
    np.savez(generated_images_str, generated_images)
    # cvae = tf.keras.models.load_model(model_str)
    plot_CVAE(cvae, test_dataset[:10], N_LATENT)

    # DCGAN
    print(f"DCGAN: {label}")
    model_str = f"{get_model_dir(DCGAN_MODEL)}{label}_{EPOCHS}_epochs_{N_LATENT}_latent_dim_model.tf"
    generated_images_str = f"{get_generated_dir(DCGAN_MODEL)}{label}_{EPOCHS}_epochs_{N_LATENT}_latent_dim_{SIZE}_n.npz"
    discriminator = get_discriminator(train_dataset.shape[1:], 1)
    generator = get_generator((N_LATENT,), train_dataset.shape[-1])
    dcgan = DCGAN(generator, discriminator, N_LATENT)
    dcgan.compile(tf.keras.optimizers.Adam(1e-4), tf.keras.optimizers.Adam(1e-4), tf.keras.losses.BinaryCrossentropy())
    dcgan.fit(train_dataset, epochs = EPOCHS, callbacks=[DCGANMonitor(num_img = 10, latent_dim = N_LATENT)], shuffle = True)
    dcgan(tf.zeros((10, N_LATENT)))
    dcgan.save(model_str, save_format="tf")
    generated_images = dcgan.generator(tf.random.normal(shape = (SIZE, N_LATENT)), training = False)
    np.savez(generated_images_str, generated_images)
    # dcgan = tf.keras.models.load_model(model_str)
    plot_DCGAN(dcgan, 10, N_LATENT)

    # CVAEDCGAN
    print(f"CVAEDCGAN: {label}")
    model_str = f"{get_model_dir(CVAEDCGAN_MODEL)}{label}_{EPOCHS}_epochs_{N_LATENT}_latent_dim_model.tf"
    generated_images_str = f"{get_generated_dir(CVAEDCGAN_MODEL)}{label}_{EPOCHS}_epochs_{N_LATENT}_latent_dim_{SIZE}_n.npz"
    encoder = get_encoder(train_dataset.shape[1:], N_LATENT)
    decoder = get_decoder((N_LATENT,), train_dataset.shape[-1])
    discriminator = get_discriminator(train_dataset.shape[1:], 1)
    cvaedcgan = CVAEDCGAN(encoder, decoder, discriminator, N_LATENT)
    cvaedcgan.compile(tf.keras.optimizers.Adam(1e-4), tf.keras.optimizers.Adam(1e-4), tf.keras.optimizers.Adam(1e-4), tf.keras.losses.BinaryCrossentropy(from_logits = True))
    cvaedcgan.fit(train_dataset, epochs = EPOCHS, callbacks=[CVAEDCGANMonitor(test_dataset, num_img = 10, latent_dim = N_LATENT)], shuffle = True)
    cvaedcgan(tf.zeros((10, *train_dataset.shape[1:])))
    cvaedcgan.save(model_str, save_format="tf")
    generated_images = cvaedcgan.decoder(tf.random.normal(shape = (SIZE, N_LATENT)), training = False)
    np.savez(generated_images_str, generated_images)
    # cvaedcgan = tf.keras.models.load_model(model_str)
    plot_CVAEDCGAN(cvaedcgan, test_dataset[:10], N_LATENT)

    # Classifier
    print(f"Classifier: {label}")
    model_str = f"{get_model_dir(CLASSIFIER_MODEL)}{label}_{EPOCHS}_epochs_{n_labels}_n_labels_model.tf"
    classifier = get_classifier(train_dataset.shape[1:], n_labels)
    classifier.compile(optimizer = tf.keras.optimizers.Adam(1e-4), loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])
    classifier.fit(train_dataset, train_labels, epochs = EPOCHS)
    classifier(tf.zeros((10, *train_dataset.shape[1:])))
    classifier.save(model_str, save_format="tf")
    test_loss, test_acc = classifier.evaluate(test_dataset,  test_labels)
    RESULTS["Classifier"]["testing_acc"][label] = test_acc * 100
    # classifier = tf.keras.models.load_model(model_str)

    # Predictions
    print(f"Predictions: {label}")
    model_str = f"{get_model_dir(CLASSIFIER_MODEL)}{label}_{EPOCHS}_epochs_{n_labels}_n_labels_model.tf"
    classifier = tf.keras.models.load_model(model_str)

    ## Geometric
    generated_images_str = generated_images_str = f"{get_generated_dir(GEOMETRIC_MODEL)}{label}_{SIZE}_n.npz"
    generated_images = tf.convert_to_tensor(np.load(generated_images_str)["arr_0"])
    predictions = classifier.predict(generated_images)
    generated_labels = [np.argmax(i) for i in predictions]
    generated_confidance = [np.max(i) for i in predictions]
    RESULTS["Predictions"]["generated_labels"][GEOMETRIC_MODEL][label] = generated_labels
    RESULTS["Predictions"]["generated_confidance"][GEOMETRIC_MODEL][label] = generated_confidance * 100

    ## CVAE
    generated_images_str = f"{get_generated_dir(CVAE_MODEL)}{label}_{EPOCHS}_epochs_{N_LATENT}_latent_dim_{SIZE}_n.npz"
    generated_images = tf.convert_to_tensor(np.load(generated_images_str)["arr_0"])
    predictions = classifier.predict(generated_images)
    generated_labels = [np.argmax(i) for i in predictions]
    generated_confidance = [np.max(i) for i in predictions]
    RESULTS["Predictions"]["generated_labels"][CVAE_MODEL][label] = generated_labels
    RESULTS["Predictions"]["generated_confidance"][CVAE_MODEL][label] = generated_confidance * 100
    
    ## DCGAN
    generated_images_str = f"{get_generated_dir(DCGAN_MODEL)}{label}_{EPOCHS}_epochs_{N_LATENT}_latent_dim_{SIZE}_n.npz"
    generated_images = tf.convert_to_tensor(np.load(generated_images_str)["arr_0"])
    predictions = classifier.predict(generated_images)
    generated_labels = [np.argmax(i) for i in predictions]
    generated_confidance = [np.max(i) for i in predictions]
    RESULTS["Predictions"]["generated_labels"][DCGAN_MODEL][label] = generated_labels
    RESULTS["Predictions"]["generated_confidance"][DCGAN_MODEL][label] = generated_confidance * 100

    ## CVAEDCGAN
    generated_images_str = f"{get_generated_dir(CVAEDCGAN_MODEL)}{label}_{EPOCHS}_epochs_{N_LATENT}_latent_dim_{SIZE}_n.npz"
    generated_images = tf.convert_to_tensor(np.load(generated_images_str)["arr_0"])
    predictions = classifier.predict(generated_images)
    generated_labels = [np.argmax(i) for i in predictions]
    generated_confidance = [np.max(i) for i in predictions]
    RESULTS["Predictions"]["generated_labels"][CVAEDCGAN_MODEL][label] = generated_labels
    RESULTS["Predictions"]["generated_confidance"][CVAEDCGAN_MODEL][label] = generated_confidance * 100




Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 14, 14, 32)   2432        ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 14, 14, 32)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 7, 7, 64)     51264       ['leaky_re_lu[0][0]']            
                                                                                            